***
*Course:* [Math 535](http://www.math.wisc.edu/~roch/mmids/) - Mathematical Methods in Data Science (MMiDS) - Fall 2020

*Name:* EDIT: write your name here 
***

# <span style="background-color:purple; color:white; padding:2px 6px">HWK 9</span> 

**Instructions:**

1. Install Julia and Jupyter notebooks by following [these instructions](https://datatofish.com/add-julia-to-jupyter/).

2. Add the necessary packages by following [these instructions](https://datatofish.com/install-package-julia/).

3. Download the required dataset from Canvas.

4. In the Jupyter notebook, write your name above and do the exercises below.

5. Run every cell.

6. "Download as" HTML. Before this step, make sure to "Restart & Run All" in the "Kernel" menu. The cell outputs should be numbered in order. 

7. Open the HTML file in a browser and save it as a PDF.

8. Due date and instructions to submit your PDF will be given on Canvas.

In [ ]:
import pandas as pd
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt

def mmids_backsubs(U,b):
    m = b.shape[0]
    x = np.zeros(m)
    for i in reversed(range(m)):
        x[i] = (b[i] - np.dot(U[i,i+1:m],x[i+1:m]))/U[i,i]
    return x

def mmids_forwardsubs(L,b):
    m = b.shape[0]
    x = np.zeros(m)
    for i in range(m):
        x[i] = (b[i] - np.dot(L[i,0:i],x[0:i]))/L[i,i]
    return x

def mmids_cholesky(B):
    n = B.shape[0] # number of rows
    L = np.zeros((n, n)) # initiallization of L
    for j in range(n):
        L[j,0:j] = mmids_forwardsubs(L[0:j,0:j],B[j,0:j])
        L[j,j] = np.sqrt(B[j,j] - LA.norm(L[j,0:j])**2)
    return L 


def wls_by_chol(A, y, w):
    W = np.diag(w)
    C = A.T @ W @ A #Computes C (Step I)
    M = mmids_cholesky(C) #Uses mmids_cholesky to compute M (Step II)
    z = mmids_forwardsubs(M, A.T @ W @ y) #Uses mmids_forwardsubs to compute z (Step III)
    return mmids_backsubs(M.T, z) #Uses mmids_backsubs to return coefficients (Step IV)

def sigmoid(z): 
    return 1/(1+np.exp(-z))

In [ ]:
# IF RUNNING ON GOOGLE COLAB
# When prompted, upload: 
#     * lebron.csv
#     * SAHeart.csv
# from your local file system
# Data source: https://www.math.wisc.edu/~roch/mmids/
# Alternative instructions: https://colab.research.google.com/notebooks/io.ipynb

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

**Background** The goal of this assignment is to implement a solution algorithm for logistic regression known as Iterative Reweighted Least Squares (IRLS) which may be preferable to gradient descent in this context, [at least for datasets of moderate dimension](https://www.youtube.com/watch?v=iwO0JPt59YQ).

We first recall some basic facts about logistic regression. Our input data is of the form $\{(\boldsymbol{\alpha}_i, b_i) : i=1,\ldots, n\}$ where $\boldsymbol{\alpha}_i \in \mathbb{R}^d$ are the features and $b_i \in \{0,1\}$ is the label. As before we use a matrix representation: $A \in \mathbb{R}^{n \times d}$ has rows $\boldsymbol{\alpha}_j^T$, $j = 1,\ldots, n$ and $\mathbf{b} = (b_1, \ldots, b_n)^T \in \{0,1\}^n$. Our goal is to find a function of the features that approximates the probability of the label $1$. For this purpose, we model the [log-odds](https://en.wikipedia.org/wiki/Logit) (or logit function) of the probability of label $1$ as a linear function of the features

$$
\log \frac{p(\boldsymbol{\alpha}; \mathbf{x})}{1-p(\boldsymbol{\alpha}; \mathbf{x})}
= \boldsymbol{\alpha}^T \mathbf{x}
$$

where $\mathbf{x} \in \mathbb{R}^d$. Inverting this expression gives
$
p(\boldsymbol{\alpha}; \mathbf{x})
= \sigma(\boldsymbol{\alpha}^T \mathbf{x})
$
where the [sigmoid](https://en.wikipedia.org/wiki/Logistic_function) function is

$$
\sigma(t)
= \frac{1}{1 + e^{-t}}
$$

for $t \in \mathbb{R}$.

We want to minimize the [cross-entropy loss](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_loss_function_and_logistic_regression) over $\mathbf{x} \in \mathbb{R}^d$

$$
\ell(\mathbf{x}; A, \mathbf{b})
= - \sum_{i=1}^n b_i \log(\sigma(\boldsymbol{\alpha}_i^T \mathbf{x}))
- \sum_{i=1}^n (1-b_i) \log(1- \sigma(\boldsymbol{\alpha}_i^T \mathbf{x})).
$$

We showed previously that the function $\ell(\mathbf{x}; A, \mathbf{b})$ is convex as a function of $\mathbf{x} \in \mathbb{R}^d$. Its gradient with respect to the parameters $\mathbf{x}$ is 

$$
\nabla_{\mathbf{x}}\,\ell(\mathbf{x}; A, \mathbf{b})
=  \sum_{i=1}^n (
\sigma(\boldsymbol{\alpha}_i^T \mathbf{x}) 
-
b_i
) \,\boldsymbol{\alpha}_i
$$

and its Hessian is

$$
\nabla^2_{\mathbf{x}} \,\ell(\mathbf{x}; A, \mathbf{b})
=  \sum_{i=1}^n \sigma(\boldsymbol{\alpha}_i^T \mathbf{x}) (1 - \sigma(\boldsymbol{\alpha}_i^T \mathbf{x}))\, \boldsymbol{\alpha}_i \boldsymbol{\alpha}_i^T
$$

where $\nabla^2_{\mathbf{x}}$ indicates the Hessian with respect to the $\mathbf{x}$ variables.

For step size $\beta$, one step of gradient descent is therefore

$$
\mathbf{x}^{k+1}
= \mathbf{x}^{k} - \beta \sum_{i=1}^n (
\sigma(\boldsymbol{\alpha}_i^T \mathbf{x}^k) - b_i
) \,\boldsymbol{\alpha}_i.
$$

**Part A: Implementing IRLS** The IRLS algorithm is based on [Newton's method](https://en.wikipedia.org/wiki/Newton%27s_method_in_optimization) which takes advantage of second-order, i.e., Hessian information to select an update direction. We only give heuristic derivation here. Let $\mathbf{x}^k$ be the current iterate and consider the quadratic approximation of $\ell(\mathbf{x}; A, \mathbf{b})$ around $\mathbf{x}^k$ obtained from the *Multivariate Taylor* expansion

$$
\ell(\mathbf{x}; A, \mathbf{b})
\approx
\ell(\mathbf{x}^k; A, \mathbf{b})
+ \nabla_{\mathbf{x}}\,\ell(\mathbf{x}; A, \mathbf{b})^T (\mathbf{x} - \mathbf{x}^k)
+ \frac{1}{2} (\mathbf{x} - \mathbf{x}^k)^T \nabla^2_{\mathbf{x}} \,\ell(\mathbf{x}^k; A, \mathbf{b}) (\mathbf{x} - \mathbf{x}^k).
$$

To choose the new iterate, we minimize this quadratic approximation. We have previously shown that the minimum is achieved at 

$$
\mathbf{x}^{k+1} - \mathbf{x}^k
= - \nabla^2_{\mathbf{x}} \,\ell(\mathbf{x}^k; A, \mathbf{b})^{-1} \nabla_{\mathbf{x}}\,\ell(\mathbf{x}; A, \mathbf{b})
$$

provided the inverse is well-defined.

To derive a more explicit expression, we first simplify the gradient and Hessian above by introducing the notation $\mathbf{z}^k = (z_1^k,\ldots,z_n^k)$ with $z_i^k = \sigma(\boldsymbol{\alpha}_i^T \mathbf{x}^k)$
and $W_k = \mathrm{diag}(z_1^k (1-z_1^k),\ldots,z_n^k (1-z_n^k))$. Then we get

$$
\nabla_{\mathbf{x}}\,\ell(\mathbf{x}^k; A, \mathbf{b})
=  \sum_{i=1}^n (
\sigma(\boldsymbol{\alpha}_i^T \mathbf{x}^k) 
-
b_i
) \,\boldsymbol{\alpha}_i
= \sum_{i=1}^n (
z_i^k
-
b_i
) \,\boldsymbol{\alpha}_i
= A^T (\mathbf{z}^k - \mathbf{b})
$$

and

$$
\nabla^2_{\mathbf{x}} \,\ell(\mathbf{x}^k; A, \mathbf{b})
=  \sum_{i=1}^n \sigma(\boldsymbol{\alpha}_i^T \mathbf{x}^k) (1 - \sigma(\boldsymbol{\alpha}_i^T \mathbf{x}^k))\, \boldsymbol{\alpha}_i \boldsymbol{\alpha}_i^T
=  \sum_{i=1}^n z_i^k (1 - z_i^k)\, \boldsymbol{\alpha}_i \boldsymbol{\alpha}_i^T
= A^T W_k A.
$$


Plugging these back into the formula for the next iterate, we arrive at

$$
\mathbf{x}^{k+1}
= \mathbf{x}^k + (A^T W_k A)^{-1} A^T (\mathbf{b} - \mathbf{z}^k).
$$

We rearrange this last expression to highlight some interesting structure. 
Note that

$$
\begin{align*}
\mathbf{x}^{k+1}
&= (A^T W_k A)^{-1}(A^T W_k A \mathbf{x}^k + A^T\mathbf{b} - A^T\mathbf{z}^k)\\
&= (A^T W_k A)^{-1} A^T W_k( A \mathbf{x}^k + W_k^{-1} (\mathbf{b} - \mathbf{z}^k))\\
&= (A^T W_k A)^{-1} A^T W_k \mathbf{y}^k
\end{align*}
$$

where we defined the working response $\mathbf{y}^k = A \mathbf{x}^k + W_k^{-1} (\mathbf{b} - \mathbf{z}^k)$.

You may recognize from a previous assignment that $\mathbf{x}^{k+1}$ is the solution to the weighted normal equations

$$
A^T W_k A \mathbf{x}^{k+1}
= A^T W_k \mathbf{y}^k.
$$

That connnection explains the name Iterative Reweighted Least Squares. You will use it below to implement the method.

**Question 1:** Complete the function below which updates the weights $w_i^k = z^k_i (1 - z_i^k)$ and the responses $y^k_i$. Use the fact that the components of $\mathbf{y}^k$ can be simplified to

$$
y_i^k 
= (A \mathbf{x}^k)_i + \frac{b_i - z_i^k}{w_i^k}.
$$

Furthermore, make use of the `sigmoid` function defined in the first code cell above.

In [ ]:
def update_w_and_y(x, A, b):
    z = #EDIT: Compute z
    w = #EDIT: Compute w
    y = #EDIT: Compute y 
    return w, y

**Question 2:** Complete the main routine of the IRLS algorithm. Make use of the `wls_by_chol` function in the first code cell above, which you will recall from a previous assignment solves the weighted least squares problem. And of course make use of `update_w_and_y`.  

In [ ]:
def mmids_irls(A, b, maxiter=int(1e2)):
    
    (n,m) = A.shape
    xk = #EDIT: Initialize x to the zero vector
    
    for _ in range(maxiter):
        wk,yk = #EDIT: Update w and y        
        xk = #EDIT: Compute next x iterate
    
    return xk

**Part B: Revisiting NBA dataset** In this part, you will re-analyze the `lebron.csv` dataset. 

**Question 3:** Load the `lebron.csv` dataset. Extract the feature `shot_distance` and the label `shot_made`. Construct the matrix $A$ and the vector $b$ as defined above. Make sure to include a column of $1$'s in A, as we have done previously. 

In [ ]:
#EDIT: Load `lebron.csv` and construct A and b as instructed

**Question 4:** Use `mmids_irls` to solve the logistic regression problem. Make sure to output your estimated $\mathbf{x}$.

In [ ]:
x_hat = #EDIT: Use IRLS to solve the logistic regression problem on A and b
print(x_hat)

**Part C: Revisiting the South African Heart Disease dataset** In this part, you will re-analyze the `SAHeart.csv` dataset.

**Question 5:** Load the `SAHeart.csv` dataset. Extract the features `sbp`, `tobacco`, `ldl`, `adiposity`, `typea`, `obesity`, `alcohol`, `age` and the label `chd`. Construct the matrix $A$ and the vector $b$ as defined above. Make sure to include a column of $1$'s in A. 

In [ ]:
#EDIT: Load `SAHeart.csv` and construct A and b as instructed

**Question 6:** Use `mmids_irls` to solve the logistic regression problem. Make sure to output your estimated $\mathbf{x}$.

In [ ]:
x_hat = #EDIT: Use IRLS to solve the logistic regression problem on A and b
print(x_hat)

**Question 7:** To get a sense of how accurate the result is, we compare our predictions to the true labels. By prediction, let us say that we mean that we predict label $1$ whenever $\sigma(\boldsymbol{\alpha}^T \mathbf{x}) > 1/2$. Compute the fraction of correct predictions on the training set using your estimated `x_hat`. 

In [ ]:
#EDIT: Compute the fraction of correct predictions